#Information
Group 9 : Resume parser

Content : Classification of resumes and getting similarity using Word2Vec

In [ ]:
# packages
import pandas as pd
import numpy as np
import seaborn as sns
import re
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import  word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/NLP_Final_changes

/content/drive/MyDrive/NLP_Final_changes


In [ ]:
ls

Bert_classification.ipynb                 sample_java_job_description.docx
Bert_Similarity.ipynb                     sample_java_resumes.csv
Category_plot.png                         sample_testing_job_description.docx
cleaned_resume_final.csv                  sample_testing_resumes.csv
Preprocessing.ipynb                       TF_IDF_Classification.ipynb
resume_classification_model/              TF_IDF_Similarity.ipynb
ResumeDataSet.csv                         Word2Vec.ipynb
sample_data_science_job_description.docx  Word_cloud.png
sample_data_science_resumes.csv


In [ ]:
# Read CSV files
df=pd.read_csv("/content/drive/MyDrive/NLP/csv/cleaned_resume_final.csv")

In [ ]:
df.head()

,Unnamed: 0,Category,Resume,cleaned_resume
0,0,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...
1,1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education detail may may data scientist data s...
2,2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...
3,3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r python sap hana tableau sap hana sql s...
4,4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education detail mca ymcaust faridabad haryana...


In [ ]:
# Remove unnamed column
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
# Labeling categories with an unique number
df['Category_no'] = df.groupby('Category').ngroup()

In [ ]:
df.head()

,Category,Resume,cleaned_resume,Category_no
0,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,5
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education detail may may data scientist data s...,5
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...,5
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r python sap hana tableau sap hana sql s...,5
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education detail mca ymcaust faridabad haryana...,5


In [ ]:
# split train and test data. Here we are using 80% train data and 20% test data. You can change based on your requirement
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.cleaned_resume,
    df.Category_no,
    test_size = 0.2,
    random_state=1,
    stratify=df.Category_no
)

In [ ]:
# Creating Word2VecTransformer class to use in the pipeline for classification
# Importing the required Word2Vec packages
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.models import Word2Vec

# This class will fit and transform the given list of strings using Word2Vec embedding
# Class Word2VecTransformer()
# Returns a word2vec object
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    # Change the vector size based on the given input
    def __init__(self, vector_size=100, window=5, min_count=1, workers=4):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.workers = workers

    # Pass a string in a list
    def fit(self, X, y=None):
        sentences = [s.split() for s in X]
        self.word2vec_model = Word2Vec(sentences,
                                       vector_size=self.vector_size,
                                       window=self.window,
                                       min_count=self.min_count,
                                       workers=self.workers)
        return self

    # Pass list of words or string of words
    def transform(self, X):
        X_transformed = []
        if isinstance(X, str):
            X = [X]
        for s in X:
            if isinstance(s, str):
                vec = [self.word2vec_model.wv[w] for w in s.split() if w in self.word2vec_model.wv]
                vec = np.mean(vec, axis=0) if vec else np.zeros(self.vector_size)
                X_transformed.append(vec)
        if not X_transformed:
            X_transformed = [np.zeros(self.vector_size)]
        return np.array(X_transformed)

In [ ]:
# import necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Using Knn for classification
clf_knn = Pipeline([
    ('word2vec', Word2VecTransformer()),
    ('KNN', KNeighborsClassifier())
])

clf_knn.fit(X_train, y_train)

# Pickle is used to save the data and can be diretly used again without running the code again
# Uncomment the below line to pickle the data after passing the required location

#pickle.dump(clf_knn, open('/content/drive/MyDrive/NLP/pkl/w2v_knn.pkl', 'wb'))

y_pred = clf_knn.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.33      0.25      0.29         4
           1       1.00      1.00      1.00         8
           2       1.00      0.60      0.75         5
           3       1.00      1.00      1.00         8
           4       0.86      1.00      0.92         6
           5       1.00      0.75      0.86         8
           6       1.00      0.86      0.92         7
           7       0.92      1.00      0.96        11
           8       0.80      1.00      0.89         8
           9       1.00      1.00      1.00         6
          10       1.00      0.78      0.88         9
          11       1.00      1.00      1.00         8
          12       1.00      1.00      1.00        17
          13       1.00      1.00      1.00         5
          14       1.00      1.00      1.00         8
          15       1.00      1.00      1.00         5
          16       1.00      1.00      1.00         8
          17       0.81    

In [ ]:
# Note: you can use your own resume to test it. But it needs to be preprocessed before passing for prediction
# Testing the model
test = []
test.append(df['Resume'][36])

y_pred_test = clf_knn.predict(test)
print('Category of sample resume is : ', df['Category'][36])
print('Category of predicted resume is : ', df.loc[df['Category_no'] == y_pred_test[0]]['Category'].unique()[0])

Category of sample resume is :  Data Science
Category of predicted resume is :  Data Science


In [ ]:
# Using Random Forest
from sklearn.ensemble import RandomForestClassifier

clf_rf = Pipeline([
    ('word2vec', Word2VecTransformer()),
    ('Random Forest', RandomForestClassifier())
])

clf_rf.fit(X_train, y_train)

# Pickle is used to save the data and can be diretly used again without running the code again
# Uncomment the below line to pickle the data after passing the required location

#pickle.dump(clf_rf, open('/content/drive/MyDrive/NLP/pkl/w2v_rf.pkl', 'wb'))

y_pred = clf_rf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.50      0.67         4
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         8
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         8
           6       1.00      1.00      1.00         7
           7       1.00      1.00      1.00        11
           8       1.00      1.00      1.00         8
           9       1.00      1.00      1.00         6
          10       1.00      1.00      1.00         9
          11       1.00      1.00      1.00         8
          12       1.00      1.00      1.00        17
          13       1.00      1.00      1.00         5
          14       1.00      1.00      1.00         8
          15       1.00      1.00      1.00         5
          16       1.00      1.00      1.00         8
          17       0.87    

In [ ]:
# Testing the model
test = []
test.append(df['Resume'][36])

y_pred_test = clf_rf.predict(test)
print('Category of sample resume is : ', df['Category'][36])
print('Category of predicted resume is : ', df.loc[df['Category_no'] == y_pred_test[0]]['Category'].unique()[0])

Category of sample resume is :  Data Science
Category of predicted resume is :  Data Science


In [ ]:
from sklearn.naive_bayes import GaussianNB
from mlxtend.preprocessing import DenseTransformer

# using Gaussian Naive Bayes
clf_gnb = Pipeline([
    ('Word2Vec', Word2VecTransformer()),
    ('to_dense', DenseTransformer()),
    ('GNB', GaussianNB())
])

clf_gnb.fit(X_train, y_train)

# Pickle is used to save the data and can be diretly used again without running the code again
# Uncomment the below line to pickle the data after passing the required location

#pickle.dump(clf_gnb, open('/content/drive/MyDrive/NLP/pkl/w2v_gnb.pkl', 'wb'))

y_pred_gnb = clf_gnb.predict(X_test)

print(classification_report(y_test, y_pred_gnb))

              precision    recall  f1-score   support

           0       0.25      0.25      0.25         4
           1       0.80      1.00      0.89         8
           2       1.00      0.60      0.75         5
           3       0.80      1.00      0.89         8
           4       0.38      0.50      0.43         6
           5       1.00      1.00      1.00         8
           6       0.86      0.86      0.86         7
           7       0.58      0.64      0.61        11
           8       0.33      0.25      0.29         8
           9       1.00      1.00      1.00         6
          10       1.00      0.78      0.88         9
          11       0.62      1.00      0.76         8
          12       0.70      0.82      0.76        17
          13       0.75      0.60      0.67         5
          14       0.67      0.75      0.71         8
          15       1.00      0.80      0.89         5
          16       0.73      1.00      0.84         8
          17       0.69    

In [ ]:
# Testing the model
test = []
test.append(df['Resume'][36])

y_pred_test = clf_gnb.predict(test)
print('Category of sample resume is : ', df['Category'][36])
print('Category of predicted resume is : ', df.loc[df['Category_no'] == y_pred_test[0]]['Category'].unique()[0])

Category of sample resume is :  Data Science
Category of predicted resume is :  Data Science


#Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#Sample data science resumes and job description

In [ ]:
# Load required CSV files
# We have taken a sample of data science resumes to compare with a sample data science job description.
# Note: You can use your own category based on your requirement of job description

df=pd.read_csv("sample_data_science_resumes.csv")

In [ ]:
# Same preprocessing function used while preprocessing the resumes
# Preprocessing the data
def preprocess_text(sentence_list):
  lines_list = []
  stop_words = set(stopwords.words('english'))
  lemmatizer = WordNetLemmatizer()

  for line in sentence_list:
      line = line.lower()
      words=[]
      for word in word_tokenize(line):
        if word.isalpha():
          if word not in stop_words:
            words.append(lemmatizer.lemmatize(word))
      lines_list.append(' '.join(words))
  
  return lines_list

In [ ]:
# add the cleaned text into the new column to save resources
df['cleaned_resume'] = preprocess_text(df['Resume'])

In [ ]:
df.head()

,Category,Resume,cleaned_resume
0,Data Science,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...
1,Data Science,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...
2,Data Science,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...
3,Data Science,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...
4,Data Science,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...


In [ ]:
# Package required to copy the data from word documents
!pip install docx2txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3977 sha256=76eb3f5c44253cb51c9cd19b7a09a2291e9ac1cdd49d2efafa572ec84d59db6d
  Stored in directory: /root/.cache/pip/wheels/40/75/01/e6c444034338bde9c7947d3467807f889123465c2371e77418
Successfully built docx2txt


In [ ]:
import docx2txt

In [ ]:
# Job description
# Get a sample job description
# Note: You can use your own job description based on requirement

job_description = docx2txt.process("sample_data_science_job_description.docx")

In [ ]:
# preprocess job description
job_description_string = preprocess_text([job_description])[0]

# Initialise word2vec object
w2v_transformer = Word2VecTransformer()
# fit transform the job description
job_desc_fit = w2v_transformer.fit([job_description_string])
job_desc_vector = w2v_transformer.transform([job_description_string])

In [ ]:
def get_similarity_scores(job_description_vector, df):
    similarity_scores = []
    
    for selected_resume in df['cleaned_resume']:
        
        # Initialise word2vec object
        w2v_transformer = Word2VecTransformer()
        # fit transform the resume
        selected_resume_fit = w2v_transformer.fit([selected_resume])
        selected_resume_vector = w2v_transformer.transform([selected_resume])

        # calculate cosine similarity
        similarity_score = cosine_similarity(selected_resume_vector, 
                                              job_desc_vector)[0][0]

        similarity_scores.append(similarity_score)
        
    return similarity_scores


In [ ]:
# Get the similarity of each resume based on job description
similarity_scores = get_similarity_scores(job_desc_vector, df)

In [ ]:
# save similarity scores
pd.set_option('mode.chained_assignment', None)
df['Similarity_Percent'] = [round(i*100,2) for i in similarity_scores]

In [ ]:
df

,Category,Resume,cleaned_resume,Similarity_Percent
0,Data Science,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,87.33
1,Data Science,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,94.48
2,Data Science,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...,87.72
3,Data Science,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...,90.84
4,Data Science,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...,81.82
5,Data Science,EDUCATION\nDINESH SAI PAPPURU\n+1(857) 313-502...,education dinesh sai pappuru boston github lin...,84.21
6,Data Science,"Nikitha Kanaparthi\nBoston, MA. 02120 • +1 (20...",nikitha kanaparthi boston linkedin github educ...,89.63
7,Data Science,VIGNAN VENNAMPALLY vennampally.v@northeastern....,vignan vennampally linkedin github education n...,88.85
8,Data Science,Hani Haider\nPhone: 781-491-3977 | Email: haid...,hani haider phone email boston linkedin github...,91.95
9,Data Science,ARTHI REDDY ALLAM\n703-509-1781 arthi@vt.edu\n...,arthi reddy allam arthi education virginia pol...,91.54


In [ ]:
# Save the sorted dataframe based on similarity
ds_sorted = df.sort_values(by='Similarity_Percent', ascending=False)

In [ ]:
# Rank the similarity based on highest similarity
ds_sorted['Predicted_Rank'] = range(1, len(ds_sorted) + 1)

In [ ]:
ds_sorted.head()

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank
16,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau data visualization r stud...,97.97,1
1,Data Science,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,94.48,2
18,Data Science,Personal Skills â¢ Ability to quickly grasp t...,personal skill ability quickly grasp technical...,93.29,3
12,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...,92.70,4
19,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,92.56,5


In [ ]:
import random

In [ ]:
# Manual ranking of the resumes which best fit the job description
# You cn manually list the ranking based on your convenience

manual_rank = [i for i in range(1, len(ds_sorted) + 1)]

In [ ]:
# Shuffle the list
random.shuffle(manual_rank)

In [ ]:
ds_sorted['Manual_Rank'] = manual_rank
ds_sorted

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank,Manual_Rank
16,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau data visualization r stud...,97.97,1,14
1,Data Science,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,94.48,2,7
18,Data Science,Personal Skills â¢ Ability to quickly grasp t...,personal skill ability quickly grasp technical...,93.29,3,6
12,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...,92.70,4,15
19,Data Science,Expertise â Data and Quantitative Analysis â...,expertise data quantitative analysis decision ...,92.56,5,18
8,Data Science,Hani Haider\nPhone: 781-491-3977 | Email: haid...,hani haider phone email boston linkedin github...,91.95,6,1
9,Data Science,ARTHI REDDY ALLAM\n703-509-1781 arthi@vt.edu\n...,arthi reddy allam arthi education virginia pol...,91.54,7,13
3,Data Science,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...,90.84,8,3
6,Data Science,"Nikitha Kanaparthi\nBoston, MA. 02120 • +1 (20...",nikitha kanaparthi boston linkedin github educ...,89.63,9,11
13,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r python sap hana tableau sap hana sql s...,89.42,10,8


In [ ]:
# Install rbo to use the rbo ranking similarity
!pip install rbo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import rbo

In [ ]:
rbo.RankingSimilarity(ds_sorted['Manual_Rank'].tolist(), ds_sorted['Predicted_Rank'].tolist()).rbo()

0.5182172535926407

#Sample java resumes and job description

In [ ]:
# Load required CSV files
# We have taken a sample of data science resumes to compare with a sample data science job description.
# Note: You can use your own category based on your requirement of job description

df=pd.read_csv("sample_java_resumes.csv")

In [ ]:
# add the cleaned text into the new column to save resources
df['cleaned_resume'] = preprocess_text(df['Resume'])

In [ ]:
df.head()

,Category,Resume,cleaned_resume
0,Java Developer,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...
1,Java Developer,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...
2,Java Developer,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...
3,Java Developer,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...
4,Java Developer,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...


In [ ]:
# Job description
# Get a sample job description
# Note: You can use your own job description based on requirement

job_description = docx2txt.process("sample_java_job_description.docx")

In [ ]:
# preprocess job description
job_description_string = preprocess_text([job_description])[0]

# Initialise word2vec object
w2v_transformer = Word2VecTransformer()
# fit transform the job description
job_desc_fit = w2v_transformer.fit([job_description_string])
job_desc_vector = w2v_transformer.transform([job_description_string])

In [ ]:
# Get the similarity of each resume based on job description
similarity_scores = get_similarity_scores(job_desc_vector, df)

In [ ]:
# save similarity scores
pd.set_option('mode.chained_assignment', None)
df['Similarity_Percent'] = [round(i*100,2) for i in similarity_scores]

In [ ]:
df.head()

,Category,Resume,cleaned_resume,Similarity_Percent
0,Java Developer,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,86.57
1,Java Developer,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,96.93
2,Java Developer,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...,91.99
3,Java Developer,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...,91.78
4,Java Developer,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...,83.24


In [ ]:
# Save the sorted dataframe based on similarity
ds_sorted = df.sort_values(by='Similarity_Percent', ascending=False)

In [ ]:
# Rank the similarity based on highest similarity
ds_sorted['Predicted_Rank'] = range(1, len(ds_sorted) + 1)

In [ ]:
ds_sorted.head()

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank
12,Java Developer,Education Details \r\nJanuary 2013 Master of E...,education detail january master engineering in...,97.98,1
13,Java Developer,"Operating Systems Windows XP, 7, 10. Tools/Pac...",operating system window xp visual basic uml pa...,97.09,2
1,Java Developer,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,96.93,3
15,Java Developer,Education Details \r\nJanuary 2016 B.E Informa...,education detail january information technolog...,95.62,4
16,Java Developer,TECHNICAL SKILLS Programming Languages: Java (...,technical skill programming language java serv...,95.18,5


In [ ]:
# Manual ranking of the resumes which best fit the job description
# You cn manually list the ranking based on your convenience

manual_rank = [i for i in range(1, len(ds_sorted) + 1)]

In [ ]:
# Shuffle the list
random.shuffle(manual_rank)

In [ ]:
ds_sorted['Manual_Rank'] = manual_rank
ds_sorted

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank,Manual_Rank
12,Java Developer,Education Details \r\nJanuary 2013 Master of E...,education detail january master engineering in...,97.98,1,1
13,Java Developer,"Operating Systems Windows XP, 7, 10. Tools/Pac...",operating system window xp visual basic uml pa...,97.09,2,14
1,Java Developer,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,96.93,3,4
15,Java Developer,Education Details \r\nJanuary 2016 B.E Informa...,education detail january information technolog...,95.62,4,5
16,Java Developer,TECHNICAL SKILLS Programming Languages: Java (...,technical skill programming language java serv...,95.18,5,21
18,Java Developer,Skills â¢ Language: Java â¢ Operating System...,skill language java operating system window li...,94.51,6,3
10,Java Developer,Technical Skills (Trained / Project Acquired S...,technical skill trained project acquired skill...,93.48,7,20
20,Java Developer,Technical Skills (Trained / Project Acquired S...,technical skill trained project acquired skill...,93.48,8,10
14,Java Developer,"Computer Skills: Languages And Script: JSP, Se...",computer skill language script jsp servlet htm...,92.97,9,7
2,Java Developer,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...,91.99,10,2


In [ ]:
rbo.RankingSimilarity(ds_sorted['Manual_Rank'].tolist(), ds_sorted['Predicted_Rank'].tolist()).rbo()

0.6783936205652261

#Sample testing resumes and job description

In [ ]:
# Load required CSV files
# We have taken a sample of data science resumes to compare with a sample data science job description.
# Note: You can use your own category based on your requirement of job description

df=pd.read_csv("sample_testing_resumes.csv")

In [ ]:
# add the cleaned text into the new column to save resources
df['cleaned_resume'] = preprocess_text(df['Resume'])

In [ ]:
df.head()

,Category,Resume,cleaned_resume
0,Testing,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...
1,Testing,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...
2,Testing,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...
3,Testing,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...
4,Testing,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...


In [ ]:
# Job description
# Get a sample job description
# Note: You can use your own job description based on requirement

job_description = docx2txt.process("sample_testing_job_description.docx")

In [ ]:
# preprocess job description
job_description_string = preprocess_text([job_description])[0]

# Initialise word2vec object
w2v_transformer = Word2VecTransformer()
# fit transform the job description
job_desc_fit = w2v_transformer.fit([job_description_string])
job_desc_vector = w2v_transformer.transform([job_description_string])

In [ ]:
# Get the similarity of each resume based on job description
similarity_scores = get_similarity_scores(job_desc_vector, df)

In [ ]:
# save similarity scores
pd.set_option('mode.chained_assignment', None)
df['Similarity_Percent'] = [round(i*100,2) for i in similarity_scores]

In [ ]:
df.head()

,Category,Resume,cleaned_resume,Similarity_Percent
0,Testing,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,85.64
1,Testing,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,96.92
2,Testing,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...,87.13
3,Testing,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...,92.31
4,Testing,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...,85.39


In [ ]:
# Save the sorted dataframe based on similarity
ds_sorted = df.sort_values(by='Similarity_Percent', ascending=False)

In [ ]:
# Rank the similarity based on highest similarity
ds_sorted['Predicted_Rank'] = range(1, len(ds_sorted) + 1)

In [ ]:
ds_sorted.head()

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank
1,Testing,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,96.92,1
11,Testing,COMPUTER PROFICIENCY â¢ Basic: MS-Office (Pow...,computer proficiency basic powerpoint word out...,95.43,2
17,Testing,COMPUTER PROFICIENCY â¢ Basic: MS-Office (Pow...,computer proficiency basic powerpoint word out...,95.43,3
16,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,95.08,4
15,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,95.08,5


In [ ]:
# Manual ranking of the resumes which best fit the job description
# You cn manually list the ranking based on your convenience

manual_rank = [i for i in range(1, len(ds_sorted) + 1)]

In [ ]:
# Shuffle the list
random.shuffle(manual_rank)

In [ ]:
ds_sorted['Manual_Rank'] = manual_rank
ds_sorted

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank,Manual_Rank
1,Testing,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,96.92,1,19
11,Testing,COMPUTER PROFICIENCY â¢ Basic: MS-Office (Pow...,computer proficiency basic powerpoint word out...,95.43,2,3
17,Testing,COMPUTER PROFICIENCY â¢ Basic: MS-Office (Pow...,computer proficiency basic powerpoint word out...,95.43,3,18
16,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,95.08,4,17
15,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,95.08,5,6
3,Testing,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...,92.31,6,16
18,Testing,â Willingness to accept the challenges. â ...,willingness accept challenge positive thinking...,91.92,7,7
13,Testing,â Willingness to accept the challenges. â ...,willingness accept challenge positive thinking...,91.92,8,1
12,Testing,Computer Skills: â¢ Proficient in MS office (...,computer skill proficient m office word basic ...,90.76,9,11
14,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...",personal skill quick learner eagerness learn n...,90.16,10,12


In [ ]:
rbo.RankingSimilarity(ds_sorted['Manual_Rank'].tolist(), ds_sorted['Predicted_Rank'].tolist()).rbo()

0.5068348619045214